# 使用 Nexus-AM 编译负载

**Build the workload using Nexus-AM**

在上一节中，我们编译得到了 XiangShan 的仿真器 `emu`，但很遗憾，我们并不能直接拿一个任意的 ELF 文件在上面运行。这是因为 XiangShan 是一个裸金属设备，不能提供操作系统、运行时库等支持。

In the previous section, we built the XiangShan emulator `emu`. However, we cannot directly run an arbitrary ELF file on it. This is because XiangShan is a bare-metal device that does not provide support such as operating systems or runtime libraries.

尽管我们可以在上面跑一个 Linux 系统，但考虑到 XiangShan 的仿真速度（~KHz），Linux 还是太过重量级，不便于快速测试和迭代。

Yes, we can run a Linux on it, but considering the simulation speed of XiangShan (~KHz), Linux is still too heavyweight and inconvenient for rapid testing and iteration.

因此，我们开发了一套裸机运行时环境 Nexus-AM，来生成无需操作系统的轻量级负载。

That's why we developed a bare-metal runtime environment called Nexus-AM to generate lightweight workloads that do not require an operating system.

`am/` 下是 nexus-am 框架的源码，`apps/` 和 `tests/` 则是常用的软件负载源码，您也可以创建自己的 app 和 test。

The `am/` directory contains the Nexus-AM framework sources; `apps/` and `tests/` hold common workload sources, and you can create your own apps and tests.

In [ ]:
%%bash
cd .. && source env.sh
cd ${AM_HOME}

tree -d -L 1

echo "apps: $(ls ./apps)"
echo "tests: $(ls ./tests)"

我们以一个简单的 Hello, XiangShan 为例，其源码位于 `${AM_HOME}/apps/hello` 目录下。我们可以直接用 `make` 来编译它：

We start with a simple "Hello, XiangShan" example, whose source code is located in the `${AM_HOME}/apps/hello` directory. We can compile it using `make`:

In [ ]:
%%bash
cd ../ && source env.sh >/dev/null
cd $AM_HOME/apps/hello

# compiling
# ARCH=riscv64-xs for XiangShan
# LINUX_GNU_TOOLCHAIN=1 to use the GNU toolchain from Ubuntu apt repo, instead of riscv64-unknown-elf
make ARCH=riscv64-xs LINUX_GNU_TOOLCHAIN=1

# check output
ls -l build

编译得到以下三个文件：

- hello-riscv64-xs.bin：程序二进制镜像（ELF 文件剥离 ELF header 等元数据），供 emu 运行使用
- hello-riscv64-xs.elf：程序的 ELF 文件
- hello-riscv64-xs.txt：程序的反汇编结果，供调试时查看

This will generate the following three files:

- hello-riscv64-xs.bin：Program binary image (The ELF header and other metadata was removed) for emu.
- hello-riscv64-xs.elf：The program's ELF file.
- hello-riscv64-xs.txt：The program’s disassembly for debugging

Nexus-AM 支持多种 ISA 和配置，您可以通过传入不同的 `ARCH` 来编译不同的目标。例如，上面使用的 `riscv64-xs` 是针对 XiangShan 的 RISC-V 64 位架构。

Nexus-AM supports multiple ISAs and configurations. You can compile for different targets by passing different `ARCH` values. For example, `riscv64-xs` used above is for XiangShan's RISC-V 64-bit architecture.

当传入的 ARCH 不支持时，`make` 会打印出所有支持的 ARCH。

If the ARCH you pass isn’t supported, `make` will print all supported ARCH values.

In [ ]:
%%bash
cd .. && source env.sh
cd ${AM_HOME}

# || true to prevent notebook erroring out on non-zero exit code
make ARCH= || true